# Forecast Model Run Collections (FMRC) with a custom Xarray Index

There are many ways one might index weather forecast output.
- "Model Run" : a single model run.
- "Constant Offset" : all values for a given lead time.
- "Constant Forecast" : all forecasts for a given time in the future.
- "Best Estimate" : A best guess stitching together the analysis or initialization fields for past forecasts with the latest forecast.

These different ways of constructing views of a forecast data are called "Forecast Model Run Collections" (FMRC). For reference, see [this classic image](https://www.unidata.ucar.edu/presentations/caron/FmrcPoster.pdf).

Assume that a data cube has been constructed with `forecast_reference_time` (commonly `time`) and `forecast_period` (commonly `step` or `lead`) as dimensions.
Then the more complex indexing patterns --- "Constant Forecast" or "Best Estimate" --- are easily achievable with numpy-style vectorized indexing.
This notebook demonstrates all 4 "FMRC" indexing patterns with a custom Xarray index.
Some complexity arises from models like HRRR where not all runs are the same length (unlike GFS).
This complexity is handled by hardcoding in what data is available for each model: for example, with HRRR we know there are 49 `step`s available every 6 hours, and 19 `steps` otherwise.

In [1]:
%load_ext autoreload

from forecast_index import (
    BestEstimate,
    ConstantForecast,
    ConstantOffset,
    ForecastIndex,
    ModelRun,
    Model,
)

## Load a data cube

In [2]:
import arraylake as al
import xarray as xr

client = al.Client()

repo = client.get_repo("earthmover-demos/hrrr")

repo

<arraylake.repo.Repo 'earthmover-demos/hrrr'>

In [3]:
repo.tree()

/
└── 📁 solar
    ├── 🇦 r2 (1799, 1059, 250, 49) float32
    ├── 🇦 t2m (1799, 1059, 250, 49) float32
    ├── 🇦 time (250,) int64
    ├── 🇦 prate (1799, 1059, 250, 49) float32
    ├── 🇦 latitude (1059, 1799) float64
    ├── 🇦 spatial_ref () int64
    ├── 🇦 dswrf (1799, 1059, 250, 49) float32
    ├── 🇦 tcc (1799, 1059, 250, 49) float32
    ├── 🇦 step (49,) int64
    ├── 🇦 longitude (1059, 1799) float64
    └── 🇦 gust (1799, 1059, 250, 49) float32

In [4]:
ds = repo.to_xarray("solar", chunks={})
ds

<xarray.Dataset> Size: 560GB
Dimensions:      (x: 1799, y: 1059, time: 250, step: 49)
Coordinates:
    latitude     (y, x) float64 15MB dask.array<chunksize=(1059, 1799), meta=np.ndarray>
    longitude    (y, x) float64 15MB dask.array<chunksize=(1059, 1799), meta=np.ndarray>
    spatial_ref  float64 8B ...
  * step         (step) timedelta64[ns] 392B 00:00:00 ... 2 days 00:00:00
  * time         (time) datetime64[ns] 2kB 2024-05-12T18:00:00 ... 2024-05-23...
Dimensions without coordinates: x, y
Data variables:
    dswrf        (x, y, time, step) float32 93GB dask.array<chunksize=(360, 120, 1, 19), meta=np.ndarray>
    gust         (x, y, time, step) float32 93GB dask.array<chunksize=(360, 120, 1, 19), meta=np.ndarray>
    prate        (x, y, time, step) float32 93GB dask.array<chunksize=(360, 120, 1, 19), meta=np.ndarray>
    r2           (x, y, time, step) float32 93GB dask.array<chunksize=(360, 120, 1, 19), meta=np.ndarray>
    t2m          (x, y, time, step) float32 93GB dask.array<chunksize=(360, 120, 1, 19), meta=np.ndarray>
    tcc          (x, y, time, step) float32 93GB dask.array<chunksize=(360, 120, 1, 19), meta=np.ndarray>
Attributes:
    description:  HRRR forecast data cube. This dataset is auto-updated by a ...

## Create a ForecastIndex

In [5]:
# Add a dummy variable to enable nice indexing syntax `.sel(forecast=ConstantForecast('2024-05-22'))`
ds.coords["forecast"] = 0

# set the new index
newds = ds.drop_indexes(["time", "step"]).set_xindex(
    ["time", "step", "forecast"], ForecastIndex, model=Model.HRRR
)
newds

<xarray.Dataset> Size: 560GB
Dimensions:      (x: 1799, y: 1059, time: 250, step: 49)
Coordinates:
    latitude     (y, x) float64 15MB dask.array<chunksize=(1059, 1799), meta=np.ndarray>
    longitude    (y, x) float64 15MB dask.array<chunksize=(1059, 1799), meta=np.ndarray>
    spatial_ref  float64 8B ...
  * time         (time) datetime64[ns] 2kB 2024-05-12T18:00:00 ... 2024-05-23...
  * step         (step) timedelta64[ns] 392B 00:00:00 ... 2 days 00:00:00
  * forecast     int64 8B 0
Dimensions without coordinates: x, y
Data variables:
    dswrf        (x, y, time, step) float32 93GB dask.array<chunksize=(360, 120, 1, 19), meta=np.ndarray>
    gust         (x, y, time, step) float32 93GB dask.array<chunksize=(360, 120, 1, 19), meta=np.ndarray>
    prate        (x, y, time, step) float32 93GB dask.array<chunksize=(360, 120, 1, 19), meta=np.ndarray>
    r2           (x, y, time, step) float32 93GB dask.array<chunksize=(360, 120, 1, 19), meta=np.ndarray>
    t2m          (x, y, time, step) float32 93GB dask.array<chunksize=(360, 120, 1, 19), meta=np.ndarray>
    tcc          (x, y, time, step) float32 93GB dask.array<chunksize=(360, 120, 1, 19), meta=np.ndarray>
Indexes:
  ┌ time      ForecastIndex
  │ step
  └ forecast
Attributes:
    description:  HRRR forecast data cube. This dataset is auto-updated by a ...

## "Standard" selection

In [6]:
newds.sel(time=slice("2024-05-20", None), step=slice("2h", '12h'))

<xarray.Dataset> Size: 38GB
Dimensions:      (x: 1799, y: 1059, time: 76, step: 11)
Coordinates:
    latitude     (y, x) float64 15MB dask.array<chunksize=(1059, 1799), meta=np.ndarray>
    longitude    (y, x) float64 15MB dask.array<chunksize=(1059, 1799), meta=np.ndarray>
    spatial_ref  float64 8B ...
  * time         (time) datetime64[ns] 608B 2024-05-20 ... 2024-05-23T03:00:00
  * step         (step) timedelta64[ns] 88B 02:00:00 03:00:00 ... 12:00:00
  * forecast     int64 8B 0
Dimensions without coordinates: x, y
Data variables:
    dswrf        (x, y, time, step) float32 6GB dask.array<chunksize=(360, 120, 1, 11), meta=np.ndarray>
    gust         (x, y, time, step) float32 6GB dask.array<chunksize=(360, 120, 1, 11), meta=np.ndarray>
    prate        (x, y, time, step) float32 6GB dask.array<chunksize=(360, 120, 1, 11), meta=np.ndarray>
    r2           (x, y, time, step) float32 6GB dask.array<chunksize=(360, 120, 1, 11), meta=np.ndarray>
    t2m          (x, y, time, step) float32 6GB dask.array<chunksize=(360, 120, 1, 11), meta=np.ndarray>
    tcc          (x, y, time, step) float32 6GB dask.array<chunksize=(360, 120, 1, 11), meta=np.ndarray>
Indexes:
  ┌ forecast  ForecastIndex
  │ time
  └ step
Attributes:
    description:  HRRR forecast data cube. This dataset is auto-updated by a ...

## FMRC indexing

In [7]:
newds.sel(forecast=BestEstimate())

<xarray.Dataset> Size: 14GB
Dimensions:      (x: 1799, y: 1059, valid_time: 298)
Coordinates:
    latitude     (y, x) float64 15MB dask.array<chunksize=(1059, 1799), meta=np.ndarray>
    longitude    (y, x) float64 15MB dask.array<chunksize=(1059, 1799), meta=np.ndarray>
    spatial_ref  float64 8B ...
    time         (valid_time) datetime64[ns] 2kB 2024-05-12T18:00:00 ... 2024...
    step         (valid_time) timedelta64[ns] 2kB 00:00:00 ... 2 days 00:00:00
  * valid_time   (valid_time) datetime64[ns] 2kB 2024-05-12T18:00:00 ... 2024...
Dimensions without coordinates: x, y
Data variables:
    dswrf        (x, y, valid_time) float32 2GB dask.array<chunksize=(360, 120, 298), meta=np.ndarray>
    gust         (x, y, valid_time) float32 2GB dask.array<chunksize=(360, 120, 298), meta=np.ndarray>
    prate        (x, y, valid_time) float32 2GB dask.array<chunksize=(360, 120, 298), meta=np.ndarray>
    r2           (x, y, valid_time) float32 2GB dask.array<chunksize=(360, 120, 298), meta=np.ndarray>
    t2m          (x, y, valid_time) float32 2GB dask.array<chunksize=(360, 120, 298), meta=np.ndarray>
    tcc          (x, y, valid_time) float32 2GB dask.array<chunksize=(360, 120, 298), meta=np.ndarray>
Attributes:
    description:  HRRR forecast data cube. This dataset is auto-updated by a ...

**TODO:** Adding a scalar `valid_time` for `ConstantForecast` triggers a bug.

In [11]:
newds.sel(forecast=ConstantForecast("2024-05-20"))

<xarray.Dataset> Size: 945MB
Dimensions:      (x: 1799, y: 1059, time: 20)
Coordinates:
    latitude     (y, x) float64 15MB dask.array<chunksize=(1059, 1799), meta=np.ndarray>
    longitude    (y, x) float64 15MB dask.array<chunksize=(1059, 1799), meta=np.ndarray>
    spatial_ref  float64 8B ...
  * time         (time) datetime64[ns] 160B 2024-05-18T06:00:00 ... 2024-05-20
    step         (time) timedelta64[ns] 160B 1 days 18:00:00 ... 00:00:00
Dimensions without coordinates: x, y
Data variables:
    dswrf        (x, y, time) float32 152MB dask.array<chunksize=(360, 120, 20), meta=np.ndarray>
    gust         (x, y, time) float32 152MB dask.array<chunksize=(360, 120, 20), meta=np.ndarray>
    prate        (x, y, time) float32 152MB dask.array<chunksize=(360, 120, 20), meta=np.ndarray>
    r2           (x, y, time) float32 152MB dask.array<chunksize=(360, 120, 20), meta=np.ndarray>
    t2m          (x, y, time) float32 152MB dask.array<chunksize=(360, 120, 20), meta=np.ndarray>
    tcc          (x, y, time) float32 152MB dask.array<chunksize=(360, 120, 20), meta=np.ndarray>
Attributes:
    description:  HRRR forecast data cube. This dataset is auto-updated by a ...

In [13]:
newds.sel(forecast=ConstantOffset("32h"))

<xarray.Dataset> Size: 2GB
Dimensions:      (x: 1799, y: 1059, time: 42)
Coordinates:
    latitude     (y, x) float64 15MB dask.array<chunksize=(1059, 1799), meta=np.ndarray>
    longitude    (y, x) float64 15MB dask.array<chunksize=(1059, 1799), meta=np.ndarray>
    spatial_ref  float64 8B ...
  * time         (time) datetime64[ns] 336B 2024-05-12T18:00:00 ... 2024-05-23
    step         timedelta64[ns] 8B 1 days 08:00:00
  * valid_time   (time) datetime64[ns] 336B 2024-05-14T02:00:00 ... 2024-05-2...
Dimensions without coordinates: x, y
Data variables:
    dswrf        (x, y, time) float32 320MB dask.array<chunksize=(360, 120, 1), meta=np.ndarray>
    gust         (x, y, time) float32 320MB dask.array<chunksize=(360, 120, 1), meta=np.ndarray>
    prate        (x, y, time) float32 320MB dask.array<chunksize=(360, 120, 1), meta=np.ndarray>
    r2           (x, y, time) float32 320MB dask.array<chunksize=(360, 120, 1), meta=np.ndarray>
    t2m          (x, y, time) float32 320MB dask.array<chunksize=(360, 120, 1), meta=np.ndarray>
    tcc          (x, y, time) float32 320MB dask.array<chunksize=(360, 120, 1), meta=np.ndarray>
Attributes:
    description:  HRRR forecast data cube. This dataset is auto-updated by a ...

In [10]:
newds.sel(forecast=ModelRun("2024-05-20 13:00"))

<xarray.Dataset> Size: 899MB
Dimensions:      (x: 1799, y: 1059, step: 19)
Coordinates:
    latitude     (y, x) float64 15MB dask.array<chunksize=(1059, 1799), meta=np.ndarray>
    longitude    (y, x) float64 15MB dask.array<chunksize=(1059, 1799), meta=np.ndarray>
    spatial_ref  float64 8B ...
    time         datetime64[ns] 8B 2024-05-20T13:00:00
  * step         (step) timedelta64[ns] 152B 00:00:00 01:00:00 ... 18:00:00
  * valid_time   (step) datetime64[ns] 152B 2024-05-20T13:00:00 ... 2024-05-2...
Dimensions without coordinates: x, y
Data variables:
    dswrf        (x, y, step) float32 145MB dask.array<chunksize=(360, 120, 19), meta=np.ndarray>
    gust         (x, y, step) float32 145MB dask.array<chunksize=(360, 120, 19), meta=np.ndarray>
    prate        (x, y, step) float32 145MB dask.array<chunksize=(360, 120, 19), meta=np.ndarray>
    r2           (x, y, step) float32 145MB dask.array<chunksize=(360, 120, 19), meta=np.ndarray>
    t2m          (x, y, step) float32 145MB dask.array<chunksize=(360, 120, 19), meta=np.ndarray>
    tcc          (x, y, step) float32 145MB dask.array<chunksize=(360, 120, 19), meta=np.ndarray>
Attributes:
    description:  HRRR forecast data cube. This dataset is auto-updated by a ...